In [1]:
import sys
sys.path.append('../')

In [2]:
from datasets import load_from_disk
from utils import prepare_tokenizer
import random

In [3]:
tkn, _ = prepare_tokenizer('../tokenizer/')
ds = load_from_disk('/root/autodl-tmp/open-orca/')
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'system_prompt', 'question', 'response'],
        num_rows: 4233923
    })
})

In [4]:
SYSTEM_MARK = '[sys]'
HUMAN_MARK = '[human]'
ROBOT_MARK = '[bot]'
ROUND_MARK_TPL = '[round:%d]'

In [6]:
ROUND_0_MARK = ROUND_MARK_TPL%(0)

def get_instruct_sep():
    rand_val = random.randint(0, 5)
    if rand_val > 0:
        return '\n' * rand_val
    return ' '

def process_line(line):
    prompt = line['system_prompt']
    if len(prompt) > 0:
        prompt = f'{SYSTEM_MARK} {prompt}{get_instruct_sep()}'
    
    que = f"{HUMAN_MARK} {line['question']}"
    resp = f"{ROBOT_MARK} {line['response']}"
    return {'text': f"{ROUND_0_MARK} {prompt}{que} {tkn.eos_token}{resp} {tkn.eos_token}"}

ds_combined = ds.map(process_line, num_proc=15)
ds_combined = ds_combined.remove_columns(['id', 'system_prompt', 'question', 'response'])

Map (num_proc=15):   0%|          | 0/4233923 [00:00<?, ? examples/s]

In [7]:
ds_combined.save_to_disk('/root/autodl-tmp/orca_parsed')

Saving the dataset (0/15 shards):   0%|          | 0/4233923 [00:00<?, ? examples/s]

In [8]:
parsed_ds = load_from_disk('/root/autodl-tmp/orca_parsed/')
parsed_ds

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 4233923
    })
})